In [ ]:
import numpy as np
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException, StaleElementReferenceException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# Khai báo trình duyệt Chrome
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# Mở URL
driver.get("https://vnexpress.net/goc-nhin")
driver.maximize_window()

def scroll_page(scroll_count=10, pause=1.5, offset=25):
    """
    Chia trang thành các lần cuộn bằng nhau, không cuộn hết mà chừa lại một đoạn nhỏ ở cuối trang.

    Args:
        scroll_count (int): Số lần cuộn.
        pause (float): Thời gian tạm dừng giữa các lần cuộn (mặc định 1 giây).
        offset (int): Độ chừa lại (tính bằng pixel) ở cuối trang.
    """
    # Lấy chiều cao tổng thể của trang
    total_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = (total_height - offset) / scroll_count  # Chiều cao mỗi lần cuộn

    for i in range(1, scroll_count + 1):
        # Cuộn xuống theo từng bước
        driver.execute_script(f"window.scrollTo(0, {i * scroll_step});")
        sleep(pause)  # Tạm dừng giữa các lần cuộn

    # print(f"Đã cuộn {scroll_count} lần, chừa lại {offset}px ở cuối trang.")


    

def close_popups():
    try:
        popup = driver.find_element(By.CSS_SELECTOR, "selector-for-popup-close-button")
        popup.click()
        sleep(1)
    except NoSuchElementException:
        pass  # No pop-up found, continue

def click_and_wait(link, retries=3):
    """
    Điều hướng đến liên kết bài viết.
    """
    for attempt in range(retries):
        try:
            # Điều hướng đến liên kết
            driver.get(link)
            sleep(2)
            return
        except Exception as e:
            print(f"Retrying navigation, attempt {attempt + 1}: {e}")
            sleep(1)
    print("Failed to navigate to the link after retries.")

#___________________________________________________________________________________________________________________________
def extract_detail_info():
    try:
        # Lấy thông tin "date"
        date_element = driver.find_element(By.CSS_SELECTOR, "span.date")
        date = date_element.text

        # Lấy thông tin "h1.title-detail"
        title_detail_element = driver.find_element(By.CSS_SELECTOR, "h1.title-detail")
        detailed_title = title_detail_element.text

        # Lấy thông tin "p.description"
        description_element = driver.find_element(By.CSS_SELECTOR, "p.description")
        detailed_description = description_element.text

        # Lấy thông tin nội dung bài viết
        content_paragraphs = driver.find_elements(By.CSS_SELECTOR, "article.fck_detail p")
        content = "\n".join([paragraph.text for paragraph in content_paragraphs])

        scroll_page(scroll_count=10, pause=1.5, offset=50)

        # Thu thập thông tin bình luận
        comments_data = []
        total_comments_count = 0
        # Nhấn vào "Xem thêm ý kiến" nhiều lần nếu tồn tại, kết hợp cuộn xuống


        # Nhấn vào "Xem thêm ý kiến" nhiều lần nếu tồn tại, kết hợp cuộn xuống
        while True:
            try:
                # Tìm nút "Xem thêm ý kiến"
                view_more_button = driver.find_element(By.CSS_SELECTOR, "div.view_more_coment a#show_more_coment")
                driver.execute_script("arguments[0].click();", view_more_button)  # Nhấn vào nút
                sleep(2)  # Đợi tải bình luận

                # Cuộn xuống để kiểm tra thêm các nút "Xem thêm ý kiến"
                driver.execute_script("window.scrollBy(0, 500);")
                sleep(1)
            except NoSuchElementException:
                # Không tìm thấy nút, thoát khỏi vòng lặp
                # print("Không tìm thấy nút 'Xem thêm ý kiến', dừng lại.")
                break

        # Sau khi cuộn và nhấn, thu thập bình luận
        comments = driver.find_elements(By.CSS_SELECTOR, "div.comment_item")
        for comment in comments:
            try:
                # Tên người bình luận
                nickname_element = comment.find_element(By.CSS_SELECTOR, "span.txt-name a.nickname")
                nickname = nickname_element.text

                # Nội dung bình luận
                comment_content_element = comment.find_element(By.CSS_SELECTOR, "p.full_content")
                comment_content = comment_content_element.text.strip()

                # Số lượt thích
                try:
                    like_element = comment.find_element(By.CSS_SELECTOR, "div.reactions-total a.number")
                    likes = like_element.text
                except NoSuchElementException:
                    # Nếu không tìm thấy số lượt thích, gán giá trị mặc định
                    likes = "0"

                # Thời gian bình luận
                time_element = comment.find_element(By.CSS_SELECTOR, "span.time-com")
                comment_time = time_element.text





        # # Nhấn vào "Xem thêm ý kiến" nhiều lần nếu tồn tại
        # while True:
        #     try:
        #         view_more_button = driver.find_element(By.CSS_SELECTOR, "div.view_more_coment a#show_more_coment")
        #         driver.execute_script("arguments[0].click();", view_more_button)
        #         sleep(2)
        #     except NoSuchElementException:
        #         # print("Không tìm thấy nút xem thêm bình luận, dừng lại.")
        #         break

        # # Sau khi cuộn và nhấn, thu thập bình luận
        # comments = driver.find_elements(By.CSS_SELECTOR, "div.comment_item")
        # for comment in comments:
        #     try:
        #         # Tên người bình luận
        #         nickname_element = comment.find_element(By.CSS_SELECTOR, "span.txt-name a.nickname")
        #         nickname = nickname_element.text

        #         # Nội dung bình luận
        #         comment_content_element = comment.find_element(By.CSS_SELECTOR, "p.full_content")
        #         comment_content = comment_content_element.text.strip()

        #         # Số lượt thích
        #         try:
        #             like_element = comment.find_element(By.CSS_SELECTOR, "div.reactions-total a.number")
        #             likes = like_element.text
        #         except NoSuchElementException:
        #             # print("Không tìm thấy số lượt thích, gán giá trị mặc định là 0.")
        #             likes = "0"

        #         # Thời gian bình luận
        #         time_element = comment.find_element(By.CSS_SELECTOR, "span.time-com")
        #         comment_time = time_element.text

                # Lấy thông tin trả lời bình luận
                reply_nicknames = []
                reply_comments = []
                reply_times = []
                reply_count = 0

                # Kiểm tra và nhấn vào nếu có reply
                try:
                    reply_button = comment.find_element(By.CSS_SELECTOR, "a.view_all_reply")
                    driver.execute_script("arguments[0].click();", reply_button)
                    sleep(1)

                    reply_items = comment.find_elements(By.CSS_SELECTOR, "div.sub_comment div.comment_item")
                    for reply in reply_items:
                        try:
                            reply_nickname_element = reply.find_element(By.CSS_SELECTOR, "span.txt-name a.nickname")
                            reply_nicknames.append(reply_nickname_element.text)

                            reply_content_element = reply.find_element(By.CSS_SELECTOR, "p.full_content")
                            reply_comments.append(reply_content_element.text.strip())

                            reply_time_element = reply.find_element(By.CSS_SELECTOR, "span.time-com")
                            reply_times.append(reply_time_element.text)
                            reply_count += 1
                        except NoSuchElementException:
                            # print("Không tìm thấy phần tử, bỏ qua trả lời này.")
                            continue
                except (NoSuchElementException, ElementClickInterceptedException):
                    # print("Không tìm thấy nút xem tất cả trả lời, bỏ qua.")
                    pass

                # Cập nhật tổng số bình luận và trả lời
                total_comments_count += 1 + reply_count

                # Lưu dữ liệu bình luận và trả lời thành các cột riêng
                comments_data.append({
                    "Nickname": nickname,
                    "Comment": comment_content,
                    "Likes": likes,
                    "Time": comment_time,
                    "Reply_Nicknames": reply_nicknames,
                    "Reply_Comments": reply_comments,
                    "Reply_Times": reply_times
                })
            except NoSuchElementException:
                # print("Không tìm thấy phần tử, bỏ qua bình luận này.")
                continue

        # Đếm tổng số bình luận và trả lời đã thu thập
        print(f"Tổng số bình luận và trả lời đã thu thập: {total_comments_count}")

        return {
            "Date": date,
            "Detailed Title": detailed_title,
            "Author's Position": detailed_description,
            "Content": content,
            "Comments_Post": comments_data,
            "Total_Comments": total_comments_count
        }
    except NoSuchElementException:
        # print("Không tìm thấy thông tin chi tiết.")
        return {}





#___________________________________________________________________________________________________________________________

def extract_articles():
    data = []
    collected_links = set()  # Tập hợp để lưu các liên kết đã thu thập
    back_count = 0  # Đếm số lần back
    retries = 0  # Đếm số lần thử lại khi không tìm thấy bài viết mới

    while len(collected_links) < 1000:  # Dừng lại sau khi xử lý 100 bài viết
        try:
            # Lấy danh sách các bài viết hiện có trên trang
            articles = driver.find_elements(By.CSS_SELECTOR, "article.item-news")
            if not articles:  # Nếu không tìm thấy bài viết nào, thử cuộn thêm
                scroll_page(scroll_count=10, pause=1.5, offset=25)
                retries += 1
                if retries > 5:  # Giới hạn số lần thử lại
                    print("Không tìm thấy bài viết mới sau 5 lần thử. Dừng lại.")
                    break
                continue

            for article in articles:
                # Lấy tiêu đề và liên kết bài viết
                try:
                    title_element = article.find_element(By.CSS_SELECTOR, "h3.title-news a")
                    title = title_element.text
                    link = title_element.get_attribute("href")

                    # Bỏ qua bài viết nếu liên kết đã được thu thập
                    if link in collected_links:
                        continue

                    # Đánh dấu bài viết là đã thu thập
                    collected_links.add(link)
                    # Hiển thị số lượng bài đã thu thập
                    print(f"Đã thu thập được {len(collected_links)} bài viết.")

                    # Lấy thông tin khác từ bài viết
                    description_element = article.find_element(By.CSS_SELECTOR, "p.description a")
                    description = description_element.text
                    try:
                        comment_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.count_cmt span.font_icon")
                        comments = comment_element.text.strip()
                    except NoSuchElementException:
                        comments = 0

                    author_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.name-author")
                    author = author_element.text
                    author_link = author_element.get_attribute("href")
                    category_element = article.find_element(By.CSS_SELECTOR, "p.meta-news a.cat")
                    category = category_element.text
                    category_link = category_element.get_attribute("href")

                    # Điều hướng đến liên kết bài viết để thu thập chi tiết
                    click_and_wait(link)
                    detail_info = extract_detail_info()

                    for comment in detail_info["Comments_Post"]:  # Tách từng bình luận
                        data.append({
                            "Title": title,
                            "Link": link,
                            "Description": description,
                            "Author": author,
                            "Author Link": author_link,
                            "Category": category,
                            "Category Link": category_link,
                            "Comments Count": comments,
                            "Date": detail_info["Date"],
                            "Author's Position": detail_info["Author's Position"],
                            "Content": detail_info["Content"],
                            "Comment Nickname": comment["Nickname"],
                            "Comment Content": comment["Comment"],
                            "Comment Likes": comment["Likes"],
                            "Reply Nicknames": comment["Reply_Nicknames"],
                            "Reply Comments": comment["Reply_Comments"],
                            "Total_Comments": detail_info["Total_Comments"]
                        })

                    # Quay lại trang chính
                    driver.back()
                    sleep(2)
                    back_count += 1

                    # Thực hiện cuộn sau mỗi 10 lần back
                    if back_count % 10 == 0:
                        scroll_page(scroll_count=10, pause=1.5, offset=25)

                except NoSuchElementException:
                    # print("Không tìm thấy phần tử trong bài viết, bỏ qua bài này.")
                    continue

            # Cuộn trang khi đã duyệt hết danh sách hiện tại
            scroll_page(scroll_count=10, pause=1.5, offset=25)

        except Exception as e:
            # print(f"Lỗi không xác định: {e}")
            scroll_page(scroll_count=10, pause=1.5, offset=25)
            continue

    print(f"Tổng số bài viết thu thập: {len(collected_links)}")
    return data


# Gọi hàm extract_articles và lưu vào DataFrame
articles_data = extract_articles()
df = pd.DataFrame(articles_data)

# In DataFrame
print(df)



Đã thu thập được 1 bài viết.
Tổng số bình luận và trả lời đã thu thập: 79
Đã thu thập được 2 bài viết.
Tổng số bình luận và trả lời đã thu thập: 18
Đã thu thập được 3 bài viết.
Tổng số bình luận và trả lời đã thu thập: 36
Đã thu thập được 4 bài viết.
Tổng số bình luận và trả lời đã thu thập: 16
Đã thu thập được 5 bài viết.
Tổng số bình luận và trả lời đã thu thập: 18
Đã thu thập được 6 bài viết.
Tổng số bình luận và trả lời đã thu thập: 13
Đã thu thập được 7 bài viết.
Tổng số bình luận và trả lời đã thu thập: 139
Đã thu thập được 8 bài viết.
Tổng số bình luận và trả lời đã thu thập: 51
Đã thu thập được 9 bài viết.
Tổng số bình luận và trả lời đã thu thập: 50
Đã thu thập được 10 bài viết.
Tổng số bình luận và trả lời đã thu thập: 38
Đã thu thập được 11 bài viết.
Tổng số bình luận và trả lời đã thu thập: 98
Đã thu thập được 12 bài viết.
Tổng số bình luận và trả lời đã thu thập: 94
Đã thu thập được 13 bài viết.
Tổng số bình luận và trả lời đã thu thập: 71
Đã thu thập được 14 bài viết.
Tổn

In [2]:
df.to_excel("D:/du lieu o cu/HUTECH Courses/Social Networking Course/SocialNetworkingProject/Project của Đạt/vnexpress_articles_1.xlsx")


In [3]:
# Lưu vào file Excel
articles_data = pd.DataFrame(articles_data)
articles_data.to_excel("D:/du lieu o cu/HUTECH Courses/Social Networking Course/SocialNetworkingProject/Project của Đạt/vnexpress_articles.xlsx")
